# Queries for Power BI

In [0]:
%sql
-- Geographic Hazard Distribution Table for Mapping
CREATE OR REPLACE TABLE hazard_map_data
USING DELTA
AS
SELECT 
    d.GEOID,
    d.state,
    d.parameter,
    d.value,
    d.hazard_level,
    d.income_bracket,
    d.median_income,
    d.result_date,
    s.tract_lat,
    s.tract_lon,
    COUNT(*) OVER (PARTITION BY d.GEOID) as incidents_per_location
FROM daily_air_quality_monitor d 
INNER JOIN sensors_with_income_levels s
    ON d.GEOID = s.GEOID;

-- Optimize the table for better query performance
OPTIMIZE hazard_map_data
ZORDER BY (state, result_date);

In [0]:
%sql
-- 1. Hazard Level Distribution by Income Bracket (Stacked Bar Chart)
CREATE OR REPLACE TABLE income_hazard_analysis
USING DELTA
AS
SELECT 
    income_bracket,
    hazard_level,
    COUNT(*) as incident_count,
    AVG(value) as avg_parameter_value,
    AVG(median_income) as avg_income,
    result_date
FROM daily_air_quality_monitor
GROUP BY income_bracket, hazard_level, result_date
ORDER BY avg_income DESC;


In [0]:
%sql
-- 2. Parameter Value Trends (Line Chart/Area Chart)
CREATE OR REPLACE TABLE parameter_value_trends
USING DELTA
AS
SELECT 
    parameter,
    income_bracket,
    result_date,
    AVG(value) as avg_value,
    MAX(value) as max_value,
    MIN(value) as min_value,
    COUNT(DISTINCT GEOID) as affected_locations
FROM daily_air_quality_monitor
GROUP BY parameter, income_bracket, result_date
ORDER BY result_date;



In [0]:
%sql
-- 3. State-Level Risk Assessment (Heat Grid/Matrix)
CREATE OR REPLACE TABLE state_risk_matrix
USING DELTA
AS
SELECT 
    state,
    hazard_level,
    income_bracket,
    COUNT(DISTINCT GEOID) as affected_areas,
    AVG(value) as avg_parameter_value,
    COUNT(*) as incident_count,
    result_date
FROM daily_air_quality_monitor
GROUP BY state, hazard_level, income_bracket, result_date
ORDER BY incident_count DESC;